In [1]:
# this notebook imports the final data set with CRSP data matched to events
# and calculates various summary stats

In [13]:
# Combine and Clean CRSP Data
from datetime import datetime
import funcy
import numpy as np
import pandas as pd
import os

from analysis.utilities.dropbox_client import DropboxAPI

DROPBOX_ANALYSIS_DATA = '/intermediate_data'
DROPBOX_CRSP = '/data/CRSP'
DATA = os.path.join(os.path.abspath(os.curdir), 'analysis/Data')
CRSP_FLAG_FNAME = 'crsp_event_flags'
TT_FNAME = 'treating_treated_ticker'

In [11]:
PIVOTS = ['PRC', 'VOL', 'RET']
TTYPES = ['TREATING', 'TREATED']
TREATING = 'TREATING'
TREATED = 'TREATED'
TREATING_DT = 'TREATING_DISCONTINUED_IKT_DATE'
TREATED_DT = 'DISCONTINUED_IKT_DATE'
FLAGS = ['SAMEMKT_SAMETECH', 'SAMEMKT_DIFFTECH', 'DIFFMKT_SAMETECH']
CRSP_LONG_FNAME = 'PRC_RET_VOL_long'
SEARCH_ONLINE = True

In [8]:
drop = DropboxAPI()

In [2]:
flag_ds1 = drop.search_and_download(CRSP_FLAG_FNAME, DROPBOX_ANALYSIS_DATA, DATA, 'p', SEARCH_ONLINE)
# drop NAT from index
flag_ds = flag_ds1[~pd.isnull(flag_ds1.index)]
del flag_ds1

NameError: name 'drop' is not defined

In [1]:
# fill all non price, volume columns with 0
to_fill = [c for c in flag_ds.columns if c.split('_')[1] not in PIVOTS]

flag_ds[to_fill] = flag_ds.loc[:, to_fill].fillna(value=0)

NameError: name 'flag_ds' is not defined

In [101]:
def count_events_in_window(series_to_count, window_plus, window_minus, counting_fnc=pd.rolling_sum):
    # WINDOW MINUS PERIOD ALWAYS INCLUDES DAY t=0, BUT WINDOW PLUS DOES NOT!!!!
    # NOTE TO AVOID DOUBLE COUNTING SAME DAY TWICE WE NEED TO SHIFT the series plus back one 
    # Count (using counting_fnc) the total events in rolling windows
    # get forward looking rolling sum by reversing series
    series_plus = counting_fnc(series_to_count[::-1], window=window_plus, min_periods=0, center=False)[::-1].shift(-1)
    # get backward looking sum
    series_minus = counting_fnc(series_to_count, window=window_minus, min_periods=0, center=False)
    # sum across series to determine if event clean
    total_events = series_minus.fillna(0).add(series_plus, fill_value=0)
#     print total_events[total_events.index > datetime(2001, 2, 5)]
    return total_events

In [102]:
def count_clean_events(event_col, is_clean_col, threshold):
    # counts the number of clean events in event_col if is_clean_col < threshold
    # if threshold is 1, that means clean events are those with 0 contaminating events
    # create dataframe merged on datetime index
    df_temp = pd.DataFrame({'events': event_col, 'is_clean': is_clean_col})
#     total_events = df_temp.loc[(df_temp['is_clean'] < threshold) & (df_temp['events'] > 0), :].shape
    total_events = df_temp.loc[(df_temp['is_clean'] < threshold) & (df_temp['events'] > 0), :]
    # clean up memory
    del df_temp
    return total_events.shape[0]
    

In [103]:
def add_ser(s1, s2):
    return s1.add(s2, fill_value=0)

def get_flag_counts(event_ser, rolling_window_df, window_plus, window_minus, threshold):
    # note pandas calls fnc 2x on first row/col to see if it can do optimizations, fnc not actually being called 2x
    is_clean_flag_df = rolling_window_df.apply(lambda x: count_events_in_window(x, window_plus, window_minus), axis=0)
    # sum across rows
    is_clean_ser = is_clean_flag_df.sum(1)
    return count_clean_events(event_ser, is_clean_ser, threshold)    

In [125]:
def total_company_fails(flag_ds, tickers):
    # Count the number of own company failures - sum number of items in 'ticker_DISC' column
    # returns series with index comp_name, values as sums
    total_counts = []
    ticker_fails = ['{0}_DISC'.format(ticker) for ticker in tickers]
    return {'Total': flag_ds.loc[:, ticker_fails].sum().sum()}

In [ ]:
def create_flag_clean_events_df(ticker, flag_ds, window_plus, window_minus, event_series_fmt, flag_series_fmts):
    # This generates the dataframe with the event series and each of rolling sums of flags to exclude, 
    # the flags themselves, one column that is a rowwise sum of all the flag columns
    # dataset useful for checking work that code is running properly
    # ticker is string of ticker
    # flag_ds is the flag dataset
    # event series fmt a string to format with the ticker string
    # flag series fmts is a list of the dirty flag formats to format with ticker name
    event_series_name = event_series_fmt.format(ticker)
    flag_series_names = map(lambda x: x.format(ticker), flag_series_fmts)
    
    # for each of the flag series, compute a rolling sum of events over the specified window. 
    flag_rolling_series = {'{}_ROLLING_SUM'.format(f_name): count_events_in_window(flag_ds.loc[:, fname], window_plus, window_minus) for f_name in flag_series_names}
    
    # make the dictionary into a dataframe, 
    # get event_series
    event_series = flag_ds.loc[:, event_series_name]
    
    
    

In [126]:
def count_own_company_exclude_CE(
    event_df, tickers, window_plus, window_minus, threshold, ttype, exclude_self=True, flags_exclude=None):
    # count own company fails (ticker_DISC) with series of other Treated confounding events 
    # flag_fails is list of flags to count when dropping treated columns
    # include_self = True means also count other own company fails in the window
    if not exclude_self and flags_exclude is None:
        print('You must either include self or supply flags to exclude')
        return
    ticker_counts = []
    for ticker in tickers:
        event_name = '{0}_DISC'.format(ticker)
        if not event_name in event_df.columns:
            print('{} has no events'.format(event_name))
            continue 
        # if event name if dataframe
        event_series = event_df.loc[:, event_name]
        excluded_colnames = []
        if flags_exclude:
            excluded_colnames = ['{0}_{1}_{2}'.format(ticker, ttype, flag_name) for flag_name in flags_exclude]
        if exclude_self:
            excluded_colnames.append('{0}_DISC'.format(ticker))
        try:
            excluded_df = event_df.loc[:, excluded_colnames]
            count = get_flag_counts(event_series, excluded_df, window_minus, window_plus, threshold)
            ticker_counts.append((ticker, count))
        except KeyError as e:
            print('None of cols exists for {}'.format(ticker))
            continue
    # Only zip together tickers that have data
    return dict(ticker_counts)

In [127]:
def count_treated_events(event_df, tickers, flags_to_count):
    # count total number of treating events by flag
    col_names = ['{0}_TREATED_{1}'.format(tick, fl) for tick in tickers for fl in flags_to_count]
    return {'Total': event_df.loc[:, col_names].sum(0).sum()}

In [128]:
count_treated_events(flag_ds, ['ABBV'], FLAGS)

{'Total': 108.0}

In [129]:
def count_treated_events_exclude_CE(
    event_df, tickers, flags_to_count, window_plus, window_minus, threshold, exclude_self=True, flags_exclude=None):
    # count treated events excluding those with confounding events in window
    # flag_exclude is list of flags to count when dropping treated columns
    # include_self = True means also count other own company fails in the window (ticker_DISC)
    # flags to count is a list of the types of treating events to count
    if not exclude_self and flags_exclude is None:
        print('You must either include self or supply flags to exclude')
        return
    ticker_counts = []
    for ticker in tickers:
        ticker_count_var = 0
        for event_flag in flags_to_count:
            # count treating events of different flags
            event_name = '{0}_{1}_{2}'.format(ticker, TREATED, event_flag)
            # Check if column exists in the data set
            if not event_name in event_df.columns:
#                 print('{} has no events'.format(event_name))
                # go to next iteration of loop
                continue
            # if event name if dataframe
            event_series = event_df.loc[:, event_name]
            excluded_colnames = []
            if flags_exclude:
                excluded_colnames = ['{0}_{1}_{2}'.format(
                        ticker, TREATED, flag_name) for flag_name in flags_exclude]
            if exclude_self:
                excluded_colnames.append('{0}_DISC'.format(ticker))
            try:
                excluded_df = event_df.loc[:, excluded_colnames]
                count = get_flag_counts(event_series, excluded_df, window_minus, window_plus, threshold)
                ticker_count_var += count
            except KeyError as e:
#                 print('None of cols exists for {}'.format(ticker))
                continue
                # for all of excluded flags, add up number of events and then 
        ticker_counts.append((ticker, ticker_count_var))
    # Only zip together tickers that have data
    return dict(ticker_counts)

In [130]:
def construct_fnc_dict_obj(flag_ds, set_tickers, window_plus, window_minus):
    return {
        'TOTAL_COMPANY_FAILURE_EVENTS': {
            'OWN_FAILURES': funcy.func_partial(
                # total company drug fail events
                total_company_fails, flag_ds, set_tickers),
            'OWN_FAILS_NO_OWN_CONFOUNDING_EVENTS': funcy.func_partial(
                # total company drug fails excluding events with same company fails in window
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, 
                window_minus, 2, TREATED, exclude_self=True, flags_exclude=None),
            'OWN_FAILS_NO_OTHER_TREATED_CONFOUNDING_EVENTS': funcy.func_partial(
                # total company fails excluding events with other treated events(of any flag type) in window
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, 
                window_minus, 2, TREATED, exclude_self=False, flags_exclude=FLAGS),
            'OWN_FAILS_NO_OWN_NO_OTHER_TREATED_CONFOUNDING_EVENTS': funcy.func_partial(
                # total company fails excluding events with other own company fails OR any other treated events of any type in window
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, 
                window_minus, 2, TREATED, exclude_self=True, flags_exclude=FLAGS),
            # Total Counts of treated events
            'TOTAL_TREATED_EVENTS_ALL': funcy.func_partial(
                count_treated_events, flag_ds, set_tickers, FLAGS),
            'TOTAL_TREATED_EVENTS_SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH']),
            'TOTAL_TREATED_EVENTS_SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events, flag_ds, set_tickers, ['SAMEMKT_SAMETECH']),
            'TOTAL_TREATED_EVENTS_DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events, flag_ds, set_tickers, ['DIFFMKT_SAMETECH']),
            # threshold one because counting all events receiving treatement excluding own company fails
            # so if any own company event occurs, treated event should be disqualified
            'TREATED_EXCLUDE_OWN_COMPANY_CE': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, 
                    FLAGS, window_plus, window_minus, 1, exclude_self=True, flags_exclude=None),
            # Counts that exclude all other treated events including target
            'TREATED_EXCLUDE_OTHER_TREATED': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, 
                    FLAGS, window_plus, window_minus, 2, exclude_self=False, flags_exclude=FLAGS),
            # Counts that exclude all other treated events including target and own company failures
            'TREATED_EVENTS_EXCLUDE_OWN_COMPANY_OTHER_TREATED': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, 
                    FLAGS, window_plus, window_minus, 2, exclude_self=True, flags_exclude=FLAGS),
        },
        'OWN_FAILS_NO_TREATED_CE_BY_FLAG': {
            # total number of company failures excluding events treated events of flag type in window
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, window_minus, 2, TREATED,
                    exclude_self=False, flags_exclude=['SAMEMKT_SAMETECH']),
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, window_minus, 2, TREATED, 
                    exclude_self=False, flags_exclude=['SAMEMKT_DIFFTECH']),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, window_minus, 2, TREATED, 
                    exclude_self=False, flags_exclude=['DIFFMKT_SAMETECH']),
        },
        'OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG': {
            # total number of company failures excluding events with same company fails in window and treated events of flag type
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, window_minus, 2, TREATED,
                    exclude_self=True, flags_exclude=['SAMEMKT_SAMETECH']),
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, window_minus, 2, TREATED, 
                    exclude_self=True, flags_exclude=['SAMEMKT_DIFFTECH']),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_own_company_exclude_CE, flag_ds, set_tickers, window_plus, window_minus, 2, TREATED, 
                    exclude_self=True, flags_exclude=['DIFFMKT_SAMETECH']),
            # BREAK BETWEEN COUNTING TREATED AND OWN COMPANY TRIGGER EVENTS
        },
        
            # count treated events of type samemkt_sametech excluding confounding events in window of 1 flag type
        'SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE': {
            # threshold is 2 because excluding same type as counted type
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_SAMETECH'], 
                window_plus, window_minus, 2, exclude_self=False, flags_exclude=['SAMEMKT_SAMETECH']),
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_SAMETECH'], 
                window_plus, window_minus, 1, exclude_self=False, flags_exclude=['SAMEMKT_DIFFTECH']),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_SAMETECH'], 
                window_plus, window_minus, 1, exclude_self=False, flags_exclude=['DIFFMKT_SAMETECH'])
        },
        'SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE': {
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'], 
                window_plus, window_minus, 1, exclude_self=False, flags_exclude=['SAMEMKT_SAMETECH']),
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'], 
                window_plus, window_minus, 2, exclude_self=False, flags_exclude=['SAMEMKT_DIFFTECH']),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'], 
                window_plus, window_minus, 1, exclude_self=False, flags_exclude=['DIFFMKT_SAMETECH'])
        },
        'DIFFMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE': {
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['DIFFMKT_SAMETECH'], 
                        window_plus, window_minus, 1, exclude_self=False, flags_exclude=['SAMEMKT_SAMETECH']),
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['DIFFMKT_SAMETECH'], 
                        window_plus, window_minus, 1, exclude_self=False, flags_exclude=['SAMEMKT_DIFFTECH']),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['DIFFMKT_SAMETECH'], 
                        window_plus, window_minus, 2, exclude_self=False, flags_exclude=['DIFFMKT_SAMETECH'])
        },
       # treated events of each type excluding own company drug failures 
        'TREATED_EXCLUDE_OWN_COMPANY_CE' :{
              'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'],
                    window_plus, window_minus, 1, exclude_self=True, flags_exclude=None),
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_SAMETECH'],
                    window_plus, window_minus, 1, exclude_self=True, flags_exclude=None),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['DIFFMKT_SAMETECH'],
                    window_plus, window_minus, 1, exclude_self=True, flags_exclude=None),
        },
            # Counts that exclude all other treated events including target
        'TREATED_EXCLUDE_OTHER_TREATED': {
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'],
                    window_plus, window_minus, 2, exclude_self=False, flags_exclude=FLAGS),
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_SAMETECH'],
                    window_plus, window_minus, 2, exclude_self=False, flags_exclude=FLAGS),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['DIFFMKT_SAMETECH'],
                    window_plus, window_minus, 2, exclude_self=False, flags_exclude=FLAGS),
        },
        # Treated events excluding all other treated and target company failures
        'TREATED_EVENTS_EXCLUDE_OWN_COMPANY_OTHER_TREATED': {
            'SAMEMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_SAMETECH'], 
                    window_plus, window_minus, 2, exclude_self=True, flags_exclude=FLAGS),
            'SAMEMKT_DIFFTECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'], 
                    window_plus, window_minus, 2, exclude_self=True, flags_exclude=FLAGS),
            'DIFFMKT_SAMETECH': funcy.func_partial(
                count_treated_events_exclude_CE, flag_ds, set_tickers, ['DIFFMKT_SAMETECH'], 
                    window_plus, window_minus, 2, exclude_self=True, flags_exclude=FLAGS),
        },
                                
    }


In [131]:
def calculate_counts(flag_df, window_plus, window_minus):
    set_tickers = set(map(lambda x: x.split('_')[0], flag_df.columns))
    counts_dict = construct_fnc_dict_obj(flag_df, set_tickers, window_plus, window_minus)
    counts_series = []
    for category, subsets in counts_dict.iteritems():
        for flag_type, fnc_obj in subsets.iteritems():
#             res = fnc_obj()
#             counts_series.append(pd.Series(data=res, name='_'.join([category, flag_type])))
            counts_series.append(pd.Series(data=fnc_obj(), name='_'.join([category, flag_type])))
    counts_df = pd.concat(counts_series, axis=1)
    sum_ser = counts_df.sum()
    # upload to csv
    drop.csv_upload_dataset(
        sum_ser, 'event_counts_plus_{0}_minus_{1}.csv'.format(
            window_plus, window_minus), DATA, DROPBOX_ANALYSIS_DATA)
    return sum_ser
    

In [132]:
calculate_counts(flag_ds, 5, 5)

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()
/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()


None of cols exists for LIFC
None of cols exists for CPD
None of cols exists for OLGC
None of cols exists for MDRX
None of cols exists for AVAN
None of cols exists for NPRM
None of cols exists for TGEN
None of cols exists for OTIC
None of cols exists for NTMD
None of cols exists for MIPI
None of cols exists for NABI
None of cols exists for TTP
None of cols exists for IGL
None of cols exists for BPAX
None of cols exists for ARCP
None of cols exists for FO
None of cols exists for MDV
None of cols exists for MMP
None of cols exists for MITI
None of cols exists for ERI
None of cols exists for NRPH
None of cols exists for TKMR
None of cols exists for STD
None of cols exists for RXII
None of cols exists for PCYO
None of cols exists for RGDO
None of cols exists for NEOT
None of cols exists for XOM
None of cols exists for TRGT
None of cols exists for CFR
None of cols exists for NUVO
None of cols exists for MBRX
None of cols exists for PABK
None of cols exists for MRX
None of cols exists for VM

OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_SAMEMKT_DIFFTECH                                339.0
OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_SAMEMKT_SAMETECH                                959.0
OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_DIFFMKT_SAMETECH                                719.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_DIFFTECH                    37.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_SAMETECH                   655.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_DIFFMKT_SAMETECH                    83.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_SAMEMKT_DIFFTECH                                       626.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_SAMEMKT_SAMETECH                                      1011.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_DIFFMKT_SAMETECH                                      1011.0
SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_DIFFTECH                 10882.0
SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_SAMETECH  

In [133]:
calculate_counts(flag_ds, 3, 3)

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=3,center=False).sum()
/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=3,center=False).sum()


None of cols exists for LIFC
None of cols exists for CPD
None of cols exists for OLGC
None of cols exists for MDRX
None of cols exists for AVAN
None of cols exists for NPRM
None of cols exists for TGEN
None of cols exists for OTIC
None of cols exists for NTMD
None of cols exists for MIPI
None of cols exists for NABI
None of cols exists for TTP
None of cols exists for IGL
None of cols exists for BPAX
None of cols exists for ARCP
None of cols exists for FO
None of cols exists for MDV
None of cols exists for MMP
None of cols exists for MITI
None of cols exists for ERI
None of cols exists for NRPH
None of cols exists for TKMR
None of cols exists for STD
None of cols exists for RXII
None of cols exists for PCYO
None of cols exists for RGDO
None of cols exists for NEOT
None of cols exists for XOM
None of cols exists for TRGT
None of cols exists for CFR
None of cols exists for NUVO
None of cols exists for MBRX
None of cols exists for PABK
None of cols exists for MRX
None of cols exists for VM

OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_SAMEMKT_DIFFTECH                                430.0
OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_SAMEMKT_SAMETECH                               1047.0
OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_DIFFMKT_SAMETECH                                828.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_DIFFTECH                    56.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_SAMETECH                   706.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_DIFFMKT_SAMETECH                   104.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_SAMEMKT_DIFFTECH                                       830.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_SAMEMKT_SAMETECH                                      1020.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_DIFFMKT_SAMETECH                                      1087.0
SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_DIFFTECH                 14240.0
SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_SAMETECH  

In [134]:
calculate_counts(flag_ds, 3, 5)

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()
/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=3,center=False).sum()


None of cols exists for LIFC
None of cols exists for CPD
None of cols exists for OLGC
None of cols exists for MDRX
None of cols exists for AVAN
None of cols exists for NPRM
None of cols exists for TGEN
None of cols exists for OTIC
None of cols exists for NTMD
None of cols exists for MIPI
None of cols exists for NABI
None of cols exists for TTP
None of cols exists for IGL
None of cols exists for BPAX
None of cols exists for ARCP
None of cols exists for FO
None of cols exists for MDV
None of cols exists for MMP
None of cols exists for MITI
None of cols exists for ERI
None of cols exists for NRPH
None of cols exists for TKMR
None of cols exists for STD
None of cols exists for RXII
None of cols exists for PCYO
None of cols exists for RGDO
None of cols exists for NEOT
None of cols exists for XOM
None of cols exists for TRGT
None of cols exists for CFR
None of cols exists for NUVO
None of cols exists for MBRX
None of cols exists for PABK
None of cols exists for MRX
None of cols exists for VM

OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_SAMEMKT_DIFFTECH                                371.0
OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_SAMEMKT_SAMETECH                                998.0
OWN_FAILS_NO_OWN_NO_TREATED_CE_BY_FLAG_DIFFMKT_SAMETECH                                769.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_DIFFTECH                    41.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_SAMETECH                   677.0
SAMEMKT_SAMETECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_DIFFMKT_SAMETECH                    95.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_SAMEMKT_DIFFTECH                                       713.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_SAMEMKT_SAMETECH                                      1016.0
OWN_FAILS_NO_TREATED_CE_BY_FLAG_DIFFMKT_SAMETECH                                      1046.0
SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_DIFFTECH                 12280.0
SAMEMKT_DIFFTECH_TREATED_EVENTS_EXCLUDE_ONE_FLAG_CE_SAMEMKT_SAMETECH  

In [89]:
count_treated_events_exclude_CE(flag_ds, ['ABBV', 'JAZZ'], ['SAMEMKT_DIFFTECH'], 5, 5, 2, False, ['SAMEMKT_DIFFTECH'])

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()
/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()


{'ABBV': 53, 'JAZZ': 68}

In [97]:

set_tickers = set(map(lambda x: x.split('_')[0], flag_ds.columns))
    
ret_dict = count_treated_events_exclude_CE(flag_ds, set_tickers, ['SAMEMKT_DIFFTECH'], 
                5, 5, 1, exclude_self=True, flags_exclude=None)
print sum(ret_dict.values())


/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()
/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=0,window=5,center=False).sum()


LIFC_TREATED_SAMEMKT_DIFFTECH has no events
CPD_TREATED_SAMEMKT_DIFFTECH has no events
OLGC_TREATED_SAMEMKT_DIFFTECH has no events
MDRX_TREATED_SAMEMKT_DIFFTECH has no events
AVAN_TREATED_SAMEMKT_DIFFTECH has no events
NPRM_TREATED_SAMEMKT_DIFFTECH has no events
TGEN_TREATED_SAMEMKT_DIFFTECH has no events
OTIC_TREATED_SAMEMKT_DIFFTECH has no events
NTMD_TREATED_SAMEMKT_DIFFTECH has no events
MIPI_TREATED_SAMEMKT_DIFFTECH has no events
NABI_TREATED_SAMEMKT_DIFFTECH has no events
TTP_TREATED_SAMEMKT_DIFFTECH has no events
IGL_TREATED_SAMEMKT_DIFFTECH has no events
BPAX_TREATED_SAMEMKT_DIFFTECH has no events
ARCP_TREATED_SAMEMKT_DIFFTECH has no events
FO_TREATED_SAMEMKT_DIFFTECH has no events
MDV_TREATED_SAMEMKT_DIFFTECH has no events
MMP_TREATED_SAMEMKT_DIFFTECH has no events
MITI_TREATED_SAMEMKT_DIFFTECH has no events
ERI_TREATED_SAMEMKT_DIFFTECH has no events
NRPH_TREATED_SAMEMKT_DIFFTECH has no events
TKMR_TREATED_SAMEMKT_DIFFTECH has no events
STD_TREATED_SAMEMKT_DIFFTECH has no even

In [93]:
tick = 'ABBV'
ticker_cols = [c for c in flag_ds.columns if tick in c]
flag_ds.loc[(flag_ds['ABBV_TREATED_SAMEMKT_DIFFTECH'] > 0) & (flag_ds['ABBV_DISC'] == 0), ticker_cols]
# flag_ds.loc[flag_ds.index > datetime(2008, 9, 30), ticker_cols]

,ABBV_DISC,ABBV_PRC,ABBV_RET,ABBV_TREATED_DIFFMKT_SAMETECH,ABBV_TREATED_SAMEMKT_DIFFTECH,ABBV_TREATED_SAMEMKT_SAMETECH,ABBV_TREATING_DIFFMKT_SAMETECH,ABBV_TREATING_SAMEMKT_DIFFTECH,ABBV_TREATING_SAMEMKT_SAMETECH,ABBV_VOL
2002-05-27,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2002-09-11,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2006-06-01,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2006-06-28,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2006-11-27,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2007-03-19,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2007-04-13,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2007-06-30,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2007-07-26,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2007-10-24,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN


In [37]:
def no_same_company_fails(event_df, tickers, window_plus, window_minus, threshold):
    # no other 'ticker_DISC' confounding events in the rolling window
    ticker_counts = []
    for ticker in tickers:
        company_events_name = '{}_DISC'.format(ticker)
        try:
            event_series = event_df.loc[:, company_events_name]
            fails = event_df.loc[:, company_events_name]
            count = get_flag_counts(event_series, [fails], window_minus, window_plus, threshold)
            ticker_counts.append((ticker, count))
        except KeyError:
#             print('{} has no events'.format(event_name))
            pass
    # Only zip together tickers that have data
    return dict(ticker_counts)

In [124]:
def no_same_flag_fails(event_df, tickers, ttype, flag, window_plus, window_minus, threshold):
    # if flag is SAMEMKT_DIFFTECH and type is treated,
    # this checks for events clean from confounding events that are TREATED_SAMEMKT_DIFFTECH and TREATING_SAMEMKT_DIFFTECH
    ticker_counts = []
    for ticker in tickers:
        event_name = '{0}_{1}_{2}'.format(ticker, ttype, flag)
        other_event_name = '{0}_{1}_{2}'.format(ticker, filter(lambda x: x != ttype, TTYPES)[0], flag)
        if event_name in event_df.columns:
            event_series = event_df.loc[:, event_name]
            if other_event_name in event_df.columns:
                fails = [event_series, event_df.loc[:, other_event_name]]
            else:
                fails = [event_series]
            count = get_flag_counts(
                event_series, fails, window_minus, window_plus, threshold)
            ticker_counts.append((ticker, count))
        else:
#             print('{} has no events'.format(event_name))
            pass
    # Only zip together tickers that have data
    return dict(ticker_counts)
    

In [27]:
flag_ds.loc[flag_ds['ABBV_TREATED_SAMEMKT_DIFFTECH'] > 0, ['ABBV_TREATED_SAMEMKT_DIFFTECH']]
# tick = 'ABBV'
# ticker_cols = [c for c in flag_ds.columns if tick in c]
# flag_ds.loc[flag_ds.index > datetime(2008, 9, 30), ticker_cols]

,ABBV_TREATED_SAMEMKT_DIFFTECH
2002-05-27,1.0
2002-09-11,1.0
2006-06-01,1.0
2006-06-28,1.0
2006-11-27,1.0
2007-03-19,1.0
2007-04-13,1.0
2007-06-30,1.0
2007-07-26,1.0
2007-10-24,1.0


In [18]:
def no_same_type_fails(event_df, tickers, ttype, flag, window_plus, window_minus, threshold):
    # if type is treated, no other news in any treated categories
    ticker_counts = []
    for ticker in tickers:
        event_name = '{0}_{1}_{2}'.format(ticker, ttype, flag)
        if event_name in event_df.columns:
            event_series = event_df.loc[:, event_name]
            fails = []
            for f in FLAGS:
                other_col = '{0}_{1}_{2}'.format(ticker, ttype, f)
                if other_col in event_df.columns:
                    fails.append(event_df.loc[:, other_col])
            count = get_flag_counts(event_series, fails, window_minus, window_plus, threshold)
            ticker_counts.append((ticker, count))
        else:
#             print('{} has no events'.format(event_name))
            pass
    # Only zip together tickers that have data
    return dict(ticker_counts)

In [19]:
def no_other_fails(event_df, tickers, ttype, flag, window_plus, window_minus, threshold):
    # no other news in any of other categories OR own company news
    ticker_counts = []
    for ticker in tickers:
        event_name = '{0}_{1}_{2}'.format(ticker, ttype, flag)
        # check to make sure series exists
        if event_name in event_df.columns:
            event_series = event_df.loc[:, event_name]
            # get other column names that contain the ticker
            other_flags = ['{0}_{1}_{2}'.format(ticker, tt_type, f) for f in FLAGS for tt_type in TTYPES]
            other_flags.append('{0}_DISC'.format(ticker))
            fail_series = []
            for col in other_flags:
                if col in event_df.columns:
                    fail_series.append(event_df.loc[:, col])
            count = get_flag_counts(event_series, fail_series, window_minus, window_plus, threshold)
            ticker_counts.append((ticker, count))
        else:
#             print('{} has no events'.format(event_name))
            pass
    # Only zip together tickers that have data
    return dict(ticker_counts)
    
    